In [19]:
import pandas as pd
import polars as pl
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict
import nflreadpy as nfl

In [20]:
draft_picks = nfl.load_draft_picks(seasons=(range(1998, 2026))).to_pandas()

In [21]:
draft_picks[draft_picks['round'] == 2]

,season,round,pick,team,gsis_id,pfr_player_id,cfb_player_id,pfr_player_name,hof,position,category,side,college,age,to,allpro,probowls,seasons_started,w_av,car_av,dr_av,games,pass_completions,pass_attempts,pass_yards,pass_tds,pass_ints,rush_atts,rush_yards,rush_tds,receptions,rec_yards,rec_tds,def_solo_tackles,def_ints,def_sacks
30,1998,2,31,OAK,None,None,None,Leon Bender,False,DT,DL,D,Washington St.,NaN,NaN,0,0,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,1998,2,32,IND,None,PathJe00,jerome-pathon-1,Jerome Pathon,False,WR,WR,O,Washington,22.0,2005.0,0,0,4,29.0,None,15.0,100.0,0.0,0.0,0.0,0.0,0.0,5.0,-7.0,0.0,260.0,3350.0,15.0,1.0,NaN,NaN
32,1998,2,33,ARI,None,ChavCo00,corey-chavous-1,Corey Chavous,False,DB,DB,D,Vanderbilt,22.0,2008.0,0,1,8,42.0,None,8.0,171.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,602.0,20.0,5.5
33,1998,2,34,TAM,None,GreeJa00,jacquez-green-1,Jacquez Green,False,WR,WR,O,Florida,22.0,2002.0,0,0,3,26.0,None,25.0,66.0,0.0,0.0,0.0,0.0,0.0,13.0,42.0,0.0,162.0,2311.0,7.0,NaN,NaN,NaN
34,1998,2,35,CHI,None,ParrTo20,tony-parrish-1,Tony Parrish,False,DB,DB,D,Washington,22.0,2006.0,0,0,8,49.0,None,24.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,516.0,30.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6940,2025,2,60,DEN,00-0040730,HarvRJ00,rj-harvey-1,RJ Harvey,False,RB,RB,O,Central Florida,24.0,2025.0,0,0,0,0.0,None,0.0,8.0,0.0,0.0,0.0,0.0,0.0,44.0,200.0,2.0,18.0,115.0,3.0,NaN,NaN,NaN
6941,2025,2,61,WAS,00-0040731,AmosTr00,trey-amos-1,Trey Amos,False,CB,DB,D,Mississippi,23.0,2025.0,0,0,0,0.0,None,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,NaN,NaN
6942,2025,2,62,CHI,00-0040741,TurnSh00,shemar-turner-1,Shemar Turner,False,DE,DL,D,Texas A&M,22.0,2025.0,0,0,0,0.0,None,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,NaN
6943,2025,2,63,KAN,00-0040748,NormOm00,omarr-norman-lott-1,Omarr Norman-Lott,False,DT,DL,D,Tennessee,23.0,2025.0,0,0,0,0.0,None,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,NaN,1.0


In [22]:
draft_picks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7138 entries, 0 to 7137
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   season            7138 non-null   int32  
 1   round             7138 non-null   int32  
 2   pick              7138 non-null   int32  
 3   team              7138 non-null   object 
 4   gsis_id           3109 non-null   object 
 5   pfr_player_id     6817 non-null   object 
 6   cfb_player_id     5882 non-null   object 
 7   pfr_player_name   7138 non-null   object 
 8   hof               7138 non-null   bool   
 9   position          7138 non-null   object 
 10  category          7138 non-null   object 
 11  side              7136 non-null   object 
 12  college           7138 non-null   object 
 13  age               6809 non-null   float64
 14  to                6463 non-null   float64
 15  allpro            7138 non-null   int32  
 16  probowls          7138 non-null   int32  


In [23]:
# draft_picks[draft_picks['gsis_id'].isna()].to_csv('draft_picks_missing_ids.csv')

In [24]:
draft_picks = draft_picks.dropna(subset='gsis_id')

In [25]:
draft_picks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3109 entries, 0 to 7137
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   season            3109 non-null   int32  
 1   round             3109 non-null   int32  
 2   pick              3109 non-null   int32  
 3   team              3109 non-null   object 
 4   gsis_id           3109 non-null   object 
 5   pfr_player_id     3109 non-null   object 
 6   cfb_player_id     2912 non-null   object 
 7   pfr_player_name   3109 non-null   object 
 8   hof               3109 non-null   bool   
 9   position          3109 non-null   object 
 10  category          3109 non-null   object 
 11  side              3107 non-null   object 
 12  college           3109 non-null   object 
 13  age               3107 non-null   float64
 14  to                2979 non-null   float64
 15  allpro            3109 non-null   int32  
 16  probowls          3109 non-null   int32  
 17  

In [26]:
draft_picks = draft_picks[['season', 'round', 'pick', 'team', 'gsis_id', 'position']]

In [27]:
draft_picks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3109 entries, 0 to 7137
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    3109 non-null   int32 
 1   round     3109 non-null   int32 
 2   pick      3109 non-null   int32 
 3   team      3109 non-null   object
 4   gsis_id   3109 non-null   object
 5   position  3109 non-null   object
dtypes: int32(3), object(3)
memory usage: 133.6+ KB


In [28]:
draft_picks['position'].value_counts()

position
WR     527
RB     391
DE     283
LB     271
DB     260
CB     251
TE     242
DT     239
QB     211
S      138
OLB     78
DL      73
K       32
P       30
SAF     30
ILB     28
FB      11
NT      11
FS       2
G        1
Name: count, dtype: int64

In [29]:
draft_picks = draft_picks.loc[draft_picks['position'].isin(['QB', 'WR', 'RB', 'TE', 'K', 'FB'])]

In [30]:
draft_picks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1414 entries, 0 to 7135
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    1414 non-null   int32 
 1   round     1414 non-null   int32 
 2   pick      1414 non-null   int32 
 3   team      1414 non-null   object
 4   gsis_id   1414 non-null   object
 5   position  1414 non-null   object
dtypes: int32(3), object(3)
memory usage: 60.8+ KB


In [31]:
# draft_picks.to_csv('draft_picks_cln.csv')

In [40]:
stats_96 = nfl.load_player_stats(seasons=list(range(1999, 2014)), summary_level= 'reg').to_pandas()


In [41]:
stats_96.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,0,None,None,None,None,None,1999,REG,SF,247,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.0,0.000000,0,13,0,0,3,18,4,5.0,28.0,1,0,0,0,0,0,11,0,0,0,0,0,0,216,1450,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,0.0,0.0
1,00-0000003,A.al-Jabbar,Abdul-Karim al-Jabbar,RB,RB,https://static.www.nfl.com/image/private/f_aut...,1999,REG,CLE,13,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,143,445,1,0,0,21,-19.765804,0,17,22,84,1,0,0,0,0,5,1.057477,0,0.0,0.038732,0.0,0.058099,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,64.9,81.9
2,00-0000007,R.Abdullah,Rabih Abdullah,RB,RB,https://static.www.nfl.com/image/private/f_aut...,1999,REG,TB,12,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,5,12,0,0,0,1,-2.231025,0,2,3,11,0,0,0,0,0,0,-2.089564,0,0.0,0.006881,0.0,0.010321,0,10,0,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,2.3,4.3
3,00-0000008,R.Abdullah,Rahim Abdullah,OLB,LB,https://static.www.nfl.com/image/private/f_aut...,1999,REG,CLE,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.0,0.000000,0,44,14,14,3,5,0,0.0,0.0,0,1,0,4,0,0,0,0,0,0,0,0,0,4,44,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,0.0,0.0
4,00-0000017,D.Abraham,Donnie Abraham,CB,DB,https://static.www.nfl.com/image/private/f_aut...,1999,REG,TB,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.0,0.000000,0,49,16,14,2,10,1,2.0,10.0,2,7,115,25,2,0,0,0,0,0,0,0,0,2,12,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,0.0,0.0


In [42]:
stats_96['position'].value_counts()

position
WR     2917
DB     2426
RB     2094
DE     2088
LB     1965
TE     1630
DT     1584
G      1396
OT     1343
OLB    1175
QB     1157
CB     1134
S       816
C       745
FS      666
K       598
P       548
FB      530
ILB     371
MLB     361
LS      354
NT      348
SAF      18
DL        3
Name: count, dtype: int64

In [43]:
stats_96 = stats_96.loc[stats_96['position'].isin(['QB', 'RB', 'WR', 'TE', 'K', 'FB'])]
stats_96.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
1,00-0000003,A.al-Jabbar,Abdul-Karim al-Jabbar,RB,RB,https://static.www.nfl.com/image/private/f_aut...,1999,REG,CLE,13,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,143,445,1,0,0,21,-19.765804,0,17,22,84,1,0,0,0,0,5,1.057477,0,0.000000,0.038732,0.000000,0.058099,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,64.90,81.90
2,00-0000007,R.Abdullah,Rabih Abdullah,RB,RB,https://static.www.nfl.com/image/private/f_aut...,1999,REG,TB,12,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,5,12,0,0,0,1,-2.231025,0,2,3,11,0,0,0,0,0,0,-2.089564,0,0.000000,0.006881,0.000000,0.010321,0,10,0,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,2.30,4.30
12,00-0000104,T.Aikman,Troy Aikman,QB,QB,https://static.www.nfl.com/image/private/f_aut...,1999,REG,DAL,14,263,441,2964,17,12,19,-130,5,1,185,2964,126,36.244714,NaN,0,16.021622,21,10,1,2,0,7,-8.846873,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,2,4,0,0,0,3,28,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,167.56,167.56
13,00-0000108,D.Akers,David Akers,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,1999,REG,PHI,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,4,0,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,0,0,0,0,3,5,2,0,53.0,0.6,0,0,0,2,1,0,0,0,0,0,2,0,53;48;46,59;51,None,147,110,0,6,6,0,0,1.0,0,0,0,0,None,0.00,0.00
19,00-0000145,D.Alexander,Derrick Alexander,WR,WR,https://static.www.nfl.com/image/private/f_aut...,1999,REG,KC,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,2,82,1,0,0,1,5.453656,0,54,105,832,2,0,0,84,0,31,9.300749,0,9.904762,0.221053,0.332016,0.563990,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,0,0,0,0,0,0,0,NaN,0,0,0,0,None,109.40,163.40


In [44]:
stats_96 = stats_96[['player_id', 'player_name', 'sacks_suffered', 'season', 'attempts', 'passing_air_yards', 'carries', 'rushing_yards', 'receptions',
                   'receiving_yards', 'fg_att', 'pat_att', 'fg_made_distance', 'fg_missed_distance']]

stats_96.head()

,player_id,player_name,sacks_suffered,season,attempts,passing_air_yards,carries,rushing_yards,receptions,receiving_yards,fg_att,pat_att,fg_made_distance,fg_missed_distance
1,00-0000003,A.al-Jabbar,0,1999,0,0,143,445,17,84,0,0,0,0
2,00-0000007,R.Abdullah,0,1999,0,0,5,12,2,11,0,0,0,0
12,00-0000104,T.Aikman,19,1999,441,185,21,10,0,0,0,0,0,0
13,00-0000108,D.Akers,0,1999,0,0,0,0,0,0,5,6,147,110
19,00-0000145,D.Alexander,0,1999,0,0,2,82,54,832,0,0,0,0


In [45]:
stats_96 = stats_96.rename({'attempts' : 'passing_att'})

In [47]:
stats_96.to_csv('stats96raw.csv')

In [48]:
stats_96['gs_unq'] = stats_96['player_id'] + '_' + stats_96['season'].astype(str)
stats_96.head()

,player_id,player_name,sacks_suffered,season,attempts,passing_air_yards,carries,rushing_yards,receptions,receiving_yards,fg_att,pat_att,fg_made_distance,fg_missed_distance,gs_unq
1,00-0000003,A.al-Jabbar,0,1999,0,0,143,445,17,84,0,0,0,0,00-0000003_1999
2,00-0000007,R.Abdullah,0,1999,0,0,5,12,2,11,0,0,0,0,00-0000007_1999
12,00-0000104,T.Aikman,19,1999,441,185,21,10,0,0,0,0,0,0,00-0000104_1999
13,00-0000108,D.Akers,0,1999,0,0,0,0,0,0,5,6,147,110,00-0000108_1999
19,00-0000145,D.Alexander,0,1999,0,0,2,82,54,832,0,0,0,0,00-0000145_1999
